In [1]:
import cv2
import numpy as np
import os
import json

In [2]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [12]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment

# YOLOv8 모델 로드
model = YOLO('yolov8x.pt')


def detect_objects_yolo(image, model):
    # 이미지를 YOLO에 맞게 변환 및 객체 탐지
    results = model(image, verbose=False)
    return results

def calculate_iou(boxA, boxB):
    # 각 박스의 좌표 (x1, y1, x2, y2) 추출
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # 교차 영역(Intersection) 계산
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 각 박스의 영역 계산
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # 합집합(Union) 영역 계산
    unionArea = boxAArea + boxBArea - interArea

    # IoU 계산
    return interArea / unionArea if unionArea > 0 else 0.0

def calculate_distance(boxA, boxB):
    # 박스 중심점 계산
    centerA = [(boxA[0] + boxA[2]) / 2, (boxA[1] + boxA[3]) / 2]
    centerB = [(boxB[0] + boxB[2]) / 2, (boxB[1] + boxB[3]) / 2]

    # 유클리드 거리 계산
    return np.sqrt((centerA[0] - centerB[0]) ** 2 + (centerA[1] - centerB[1]) ** 2)

def calculate_classwise_iou_by_distance(prev_boxes, curr_boxes):
    # 클래스별로 객체 분류
    prev_class_map = {}
    for box in prev_boxes:
        cls = int(box.cls)
        if cls not in prev_class_map:
            prev_class_map[cls] = []
        prev_class_map[cls].append(box.xyxy[0].cpu().numpy())

    curr_class_map = {}
    for box in curr_boxes:
        cls = int(box.cls)
        if cls not in curr_class_map:
            curr_class_map[cls] = []
        curr_class_map[cls].append(box.xyxy[0].cpu().numpy())

    # IoU 계산을 위한 변수 초기화
    total_iou = 0.0
    pair_count = 0

    # 이전 프레임과 현재 프레임에 동일하게 존재하는 클래스만 처리
    for cls in prev_class_map.keys() & curr_class_map.keys():  # 클래스 종류가 동일한 클래스만 선택
        prev_class_boxes = prev_class_map[cls]
        curr_class_boxes = curr_class_map[cls]

        # 두 프레임의 해당 클래스에 객체가 존재하지 않으면 건너뛰기
        if len(prev_class_boxes) == 0 or len(curr_class_boxes) == 0:
            continue

        # 이전 프레임과 현재 프레임의 동일 클래스 객체 간 유클리드 거리 기반 일대일 매칭 수행
        distance_matrix = np.zeros((len(prev_class_boxes), len(curr_class_boxes)))
        for i, prev_box in enumerate(prev_class_boxes):
            for j, curr_box in enumerate(curr_class_boxes):
                distance_matrix[i, j] = calculate_distance(prev_box, curr_box)

        # Hungarian 알고리즘을 사용하여 최소 거리 기반으로 매칭 수행
        row_ind, col_ind = linear_sum_assignment(distance_matrix)  # 최소값 기준으로 최적화
        matched_pairs = [(prev_class_boxes[i], curr_class_boxes[j]) for i, j in zip(row_ind, col_ind)]

        # 매칭된 짝들의 IoU 계산
        matched_ious = [calculate_iou(pair[0], pair[1]) for pair in matched_pairs]

        # IoU 평균 계산
        total_iou += sum(matched_ious)
        pair_count += len(matched_ious)

    # 전체 매칭된 객체들의 평균 IoU 반환
    return total_iou / pair_count if pair_count > 0 else 0.0

# 히스토그램 계산 함수
def calculate_histograms(image, bins=256):
    channels = cv2.split(image)
    histograms = [cv2.normalize(cv2.calcHist([ch], [0], None, [bins], [0, 256]), None).flatten() for ch in channels]
    return histograms
    
# RGB image를 display하는 함수
def show_rgb_image(img, title=None):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # Convert BGR to RGB
    plt.axis('off')
    if title:
        plt.title(title)
    plt.show()
    
# 샷 경계 검출
def shot_boundary_detection_with_yolo(folder_path, model, threshold=0.8, iou_threshold=0.5):
    images = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')])

    prev_image = cv2.imread(images[0])
    prev_hists = calculate_histograms(prev_image)
    
    shot_boundaries = []
    
    for idx, image_path in enumerate(images[1:], 1):
        current_image = cv2.imread(image_path)
        curr_hists = calculate_histograms(current_image)

        # 히스토그램 유사도 계산 (Correlation 방식)
        similarities = [cv2.compareHist(prev_hists[i], curr_hists[i], cv2.HISTCMP_CORREL) for i in range(3)]
        
        # 유사도가 threshold 미만일 경우에만 YOLO 객체 검출 수행
        if min(similarities) < threshold:
            prev_objects = detect_objects_yolo(prev_image, model)
            curr_objects = detect_objects_yolo(current_image, model)

            # 이전 및 현재 프레임의 객체 바운딩 박스 추출 및 클래스별 IoU 계산
            iou = calculate_classwise_iou(prev_objects[0].boxes, curr_objects[0].boxes)
            
            # IoU가 iou_threshold 미만일 경우 샷 경계로 간주
            if iou < iou_threshold and iou != 0.0:
                shot_boundaries.append(f"frame_{str(idx).zfill(5)}")
            
        prev_image = current_image  # 이전 프레임 이미지 갱신
        prev_hists = curr_hists  # 이전 프레임 히스토그램 갱신

    return shot_boundaries


In [16]:
#테스트 결과 출력
import os

def calculate_metrics(shot_boundaries, ground_truth):
    tp = [frame for frame in shot_boundaries if frame in ground_truth]
    fp = [frame for frame in shot_boundaries if frame not in ground_truth]
    fn = [frame for frame in ground_truth if frame not in shot_boundaries]

    true_positives = len(tp)
    false_positives = len(fp)
    false_negatives = len(fn)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score, tp, fp, fn

def process_videos(base_folder, annotations_folder, histogram_threshold, iou_threshold):
    results = []
    for video_category in os.listdir(base_folder):
        category_path = os.path.join(base_folder, video_category)
        annotation_category_path = os.path.join(annotations_folder, video_category)

        if os.path.isdir(category_path):
            for video_id in os.listdir(category_path):
                video_path = os.path.join(category_path, video_id)
                annotation_path = os.path.join(annotation_category_path, f"{video_id}.txt")
                
                # YOLOv8 모델 로드
                model = YOLO('yolov8x.pt')

                # Shot boundary detection 수행
                shot_boundaries = shot_boundary_detection_with_yolo(video_path, model, histogram_threshold, iou_threshold)

                # Ground truth txt 파일 읽기
                with open(annotation_path, 'r') as f:
                    ground_truth = [line.strip() for line in f.readlines()]

                # Precision, Recall, F1 계산 및 tp, fp, fn 프레임 파일명 저장
                precision, recall, f1_score, tp, fp, fn = calculate_metrics(shot_boundaries, ground_truth)

                # 결과 저장
                results.append({
                    "video_category": video_category,
                    "video_id": video_id,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1_score,
                    "false_positives": fp,  # FP에 해당하는 프레임 파일명 리스트
                    "false_negatives": fn   # FN에 해당하는 프레임 파일명 리스트
                })

                print(f"Category: {video_category}, Video: {video_id}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
                print(f"FP: {fp}")
                print(f"FN: {fn}")
                print('\n')
    
    return results

# 사용 예시
base_folder = "frames"
annotations_folder = "annotations"
histogram_threshold = 0.8
iou_threshold = 0.55

# 모든 폴더와 파일을 순회하며 계산
results = process_videos(base_folder, annotations_folder, histogram_threshold, iou_threshold)

Category: darling, Video: 1, Precision: 1.0000, Recall: 0.9545, F1 Score: 0.9767
FP: []
FN: ['frame_00463', 'frame_01822']


Category: darling, Video: 2, Precision: 1.0000, Recall: 0.9787, F1 Score: 0.9892
FP: []
FN: ['frame_02173']


Category: darling, Video: 3, Precision: 0.9545, Recall: 0.7000, F1 Score: 0.8077
FP: ['frame_02695']
FN: ['frame_00009', 'frame_01214', 'frame_01819', 'frame_02622', 'frame_03168', 'frame_03902', 'frame_04049', 'frame_04196', 'frame_04306']


Category: hypeboy, Video: 1, Precision: 0.9783, Recall: 0.9783, F1 Score: 0.9783
FP: ['frame_00219']
FN: ['frame_05533']


Category: hypeboy, Video: 2, Precision: 1.0000, Recall: 0.9808, F1 Score: 0.9903
FP: []
FN: ['frame_01875']


Category: hypeboy, Video: 3, Precision: 0.8261, Recall: 0.9744, F1 Score: 0.8941
FP: ['frame_01435', 'frame_01566', 'frame_02444', 'frame_03330', 'frame_03452', 'frame_03453', 'frame_04155', 'frame_04159']
FN: ['frame_05500']


Category: readytolove, Video: 1, Precision: 0.9483, Recall: 0